In [1]:
import os
import pickle

import matplotlib.pyplot as plt

import numpy as np
import pymc3
import theano.tensor as tt

In [2]:
KB = 0.0019872041        # in kcal/mol/K
INJ_VOL = 1.2e-5         # in liter
CELL_CONCENTR = 0.1      # milli molar
SYRINGE_CONCENTR = 1.0   # milli molar

HEAT_FILE = "inputs/Mg1EDTAp1a.DAT"
OUT_DIR = "outputs"

In [ ]:
def equilibrium_concentrations(Kd1, Kd2, C0_R, C0_L1, C0_L2, V):
    """
    REF: Zhi-Xin Wang An exact mathematical expression for describing competitive
                        binding of two different ligands to a protein molecule
                        FEBS Letters 360 (1995) 111-114

    Kd1, Kd2        :   disociation constants of ligand 1 and 2, respectively (M)
    C0_R            :   total concentration receptor   (M)
    C0_L1, C0_L2    :   total concentrations of ligand 1 and ligand 2, respectively (M)
    V               :   volume of sample cell (L)
    return
                [RL1, RL2]  complex concentrations (M)
    """

    a = Kd1 + Kd2 + C0_L1 + C0_L2 - C0_R

    b = Kd2*(C0_L1 - C0_R) + Kd1*(C0_L2 - C0_R) + Kd1*Kd2

    c = -Kd1 * Kd2 * C0_R

    d = tt.sqrt(a*a - 3*b)

    e = tt.clip((-2. * a**3 + 9. * a * b - 27. * c) / (2. * d**3), -1, 1)

    theta = tt.arccos(e)

    RL1 = C0_L1 * (2. * d * tt.cos(theta/3.) - a) / (3. * Kd1 + (2. * d * tt.cos(theta/3.) - a))
    RL2 = C0_L2 * (2. * d * tt.cos(theta/3.) - a) / (3. * Kd2 + (2. * d * tt.cos(theta/3.) - a))

    return RL1, RL2
